In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from scipy import stats
import warnings
warnings.filterwarnings(action='ignore')

## Load Data

In [56]:
data_url = "./sensor_data_temp.log"
df = pd.read_csv(data_url, sep=" ", header=None, index_col=None)
df.head(1)

,0,1,2,3,4
0,1022-09-14,19:33:07,T=22.0,H=20.0,TO=45


In [57]:
class CheckDataQuality:
	def __init__(self, dataframe):
		self.dataframe = dataframe
	
	valid_temp = [-20, 50]  # Wertebereich Temperatur
	valid_hum = [0, 100]    # Wertebereich Luftfeuchtigkeit
	  
	def rename_columns(self):
		""" rename_columns(df)
		Renames the columns of the Dataset to ['Date', 'Time', 'Temp', 'Hum', 'TO']
	
		:param:     df
		:returns:   a boolean value
		:rtype:     bool
		"""
		try:
			self.dataframe.columns = ['Date', 'Time', 'Temp', 'Hum', 'TO']
			return True
		except:
			return False

	def create_datetime(self):
		""" create_datetime(df)
		Creates a new column 'Datetime' from 'Date' and 'Time'. Drops columns 'Date' and 'Time'.
	
		:param:     df
		:returns:   a boolean value
		:rtype:     bool
		"""
		try:
			# Converting colums Date + Time to new column with pd.timestamp 'Datetime'
			self.dataframe['Datetime'] = pd.to_datetime(self.dataframe['Date'] + ' ' + self.dataframe['Time'], errors="coerce")
			# Dropping Columns Date and Time
			self.dataframe = self.dataframe.drop(columns=['Date', 'Time'])
			return True
		except:
			return False

	def get_first_valid_timestamp(self):
		try:
			first_index = 0
			print("*** Entering get_first_valid_timestamp ***")

			# looking for first valid timestamp
			for index in self.dataframe.index:
				start_time = self.dataframe['Datetime'][first_index]
				if np.isnat(np.datetime64(str(start_time))):
					first_index+=1
				else:
					first_index = first_index
			print(f"First valid Timestamp: {start_time}, index: {first_index}")

			return [start_time, first_index]
		except:
			return [0, 0]

	def get_last_valid_timestamp(self):
		try:
			last_index = len(self.dataframe['Datetime'])-1
			end_time = self.dataframe['Datetime'][last_index]

			# looking for last valid timestamp
			for index in self.dataframe.index:
				end_time = self.dataframe['Datetime'][last_index]
				if np.isnat(np.datetime64(str(end_time))):
					last_index-=1
				else:
					last_index = last_index
			print(f"Last valid Timestamp: {end_time}, index: {last_index}")

			return [end_time, last_index]
		except:
			return [0, 0]

	def calculate_mean_timegap(start_time, end_time, first_index, last_index):
		try:

			# calculating mean timegap between timestamps
			mean_timegap = (end_time - start_time) / (last_index - first_index)
			mean_timegap = pd.to_timedelta(str(mean_timegap)).round('1s')
			print(f"Mean Timegap: {mean_timegap}")

			return [mean_timegap]
		except:
			return [0, 0]    


	def replace_nat(self):
		""" replace_nat(df, mean_timegap)
		Checks the dataframe for NaT. Replaces NaT with calculated Timestamp values.

		:param:     df = dataframe, mean_timegap = pandas datetime with "%Y-%m-%d %H:%M:%S"
		:returns:   a boolean value
		:rtype:     bool
		"""    
		try:
			# first_index = 0
			# last_index = len(self.dataframe['Datetime'])-1
			# end_time = self.dataframe['Datetime'][last_index]
			nat_index = []

			# # looking for first valid timestamp
			# for index in self.dataframe.index:
			#     start_time = self.dataframe['Datetime'][first_index]
			#     if np.isnat(np.datetime64(str(start_time))):
			#         first_index+=1
			#     else:
			#         first_index = first_index
			# print(f"First valid Timestamp: {start_time}, index: {first_index}")

			# # looking for last valid timestamp
			# for index in self.dataframe.index:
			#     end_time = self.dataframe['Datetime'][last_index]
			#     if np.isnat(np.datetime64(str(end_time))):
			#         last_index-=1
			#     else:
			#         last_index = last_index
			# print(f"Last valid Timestamp: {end_time}, index: {last_index}")

			# # calculating mean timegap between timestamps
			# mean_timegap = (end_time - start_time) / (last_index - first_index)
			# mean_timegap = pd.to_timedelta(str(mean_timegap)).round('1s')
			# print(f"Mean Timegap: {mean_timegap}")

			# Get first valid timestamp
			start_time = get_first_valid_timestamp(self.dataframe)[0]
			first_index = get_first_valid_timestamp(self.dataframe)[1]

			# Get last valid timestamp
			end_time = get_last_valid_timestamp(self.dataframe)[0]
			last_index = get_last_valid_timestamp(self.dataframe)[1]
			
			# Calculate mean timegap
			mean_timegap = calculate_mean_timegap(start_time, end_time, first_index, last_index)

			# checking for NaT
			print("Checking for NaT...")
			for index in self.dataframe.index:
				if np.isnat(np.datetime64(str(self.dataframe['Datetime'][index]))):
					print("df[" + str(index) + "]['Datetime']=" + str(self.dataframe['Datetime'][index]) + ": " + str(np.isnat(np.datetime64(str(self.dataframe['Datetime'][index])))))
					nat_index.append(index)
			print(f"Indices with NaT: {nat_index}")
				
			# replacing NaT with calculated Timestamps
			print("Replacing NaT with calculated Timestamps...")
			#for nat in reversed(nat_index):
			for nat in nat_index[::-1]:
				if (first_index < nat):
					if not np.isnat(np.datetime64(str(self.dataframe['Datetime'][nat - 1]))):
						self.dataframe['Datetime'][nat] = self.dataframe['Datetime'][nat - 1] + mean_timegap
					else: 
						self.dataframe['Datetime'][nat] = self.dataframe['Datetime'][nat + 1] - mean_timegap
				elif (first_index >= nat):
					self.dataframe['Datetime'][nat] = self.dataframe['Datetime'][first_index] - mean_timegap
					if first_index > 1:
						first_index-=1
				else: 
					self.dataframe['Datetime'][nat] = self.dataframe['Datetime'][first_index] - mean_timegap
			
				print("Calculated Timestamp for: df[" + str(nat) + "]['Datetime']=" + str(self.dataframe['Datetime'][nat]))
			
			print("Replacing NaT done!")
			return True                  
		except:
			print("Warning: Something went wrong!")
			return False
		
	def check_valid_date(self):
		""" check_valid_date(self)
		Checks if the date is valid and replaces invalid dates with NaT. Calls replace_nat function to replace NaT with calculated Timestamp. 

		:param:     df
		:returns:   a boolean value
		:rtype:     bool
		""" 
		first_index = 0
		now = datetime.today()

		try:
			# Looking for first valid timestamp
			for index in self.dataframe.index:
				start_time = self.dataframe['Datetime'][first_index]
				if np.isnat(np.datetime64(str(start_time))) and not (start_time <= now):
					first_index+=1
				else:
					first_index = first_index
					last_date = (self.dataframe['Datetime'][first_index])
			# print(f"First valid Timestamp: {start_time}, index: {first_index}, last_date: {last_date}")
				
			# Checking for valid dates and replacing invalid dates with NaT
			for index in self.dataframe.index:
				
				if first_index >= index:
					last_date = (self.dataframe['Datetime'][first_index])
				elif first_index == 0:
					last_date = 0
				else: 
					last_date = (self.dataframe['Datetime'][index-1])

				actual_date = (self.dataframe['Datetime'][index])

				next_date = (self.dataframe['Datetime'][index+1])
				
				# Checking if Datetime is allready NaT
				if np.isnat(np.datetime64(str(self.dataframe['Datetime'][index]))):
					is_nat = True
				else:
					is_nat = False           
		   
				# Change Datetime to NaT if (start_time <= datecheck <= now), if actual_date >= next_date, if duplicate
				if not is_nat:
					if (not (start_time <= actual_date <= now)) or (actual_date >= next_date):             
						print("df[" + str(index) + "]['Datetime']=" + str(self.dataframe['Datetime'][index]) + " is not valid! Changing invalid Date to NaT. Last Date: " + str(last_date))
						# Changing invalid Date to NaT
						#self.dataframe['Datetime'][index] = str("NaT")
						self.dataframe.loc[self.dataframe.Datetime == index] = str("NaT")
						
		#     #print("Checking for valid dates done!")
		#     return True
		# except:
		#     #print("Warning: Checking for valid dates was not succesfull!")
		#     return False
		except Exception as e:
			print(f'EXCEPTION - Something strange is going on: {type(e)}, Index: {index}')

	def change_index(self):
		try:
			print("Setting Datetime as index...")
			self.dataframe.set_index(pd.DatetimeIndex(df['Datetime'], inplace=True))
			print("Reindexing columns...")
			column_names = ["Datetime", "Temp", "Hum", "TO"]
			self.dataframe = self.dataframe.reindex(columns=column_names)
			return True
		except:
			return False
			   
	# Format Data Columns
	def format_data_columns(self):
		try:
			# Replacing Strings in Temp and Hum, Dropping TO
			print("Formatting Data Columns...")
			print("String Extraction...")
			self.dataframe['Temp'] = self.dataframe['Temp'].str.replace('T=', '')
			self.dataframe['Hum'] = self.dataframe['Hum'].str.replace('H=', '')
			print("Dropping Column TO...")
			self.dataframe = self.dataframe.drop(columns=['TO'])
			# Convert each value of the column to a string
			print("Converting from string to float...")
			self.dataframe['Temp'] = pd.to_numeric(self.dataframe['Temp'], errors='coerce')
			self.dataframe['Hum'] = pd.to_numeric(self.dataframe['Hum'], errors='coerce')
			# Replace empty string ('') with np.nan before convertion
			self.dataframe['Temp'] = self.dataframe['Temp'].replace(r'^\s*$', np.nan, regex=True)
			self.dataframe['Hum'] = self.dataframe['Hum'].replace(r'^\s*$', np.nan, regex=True)
			print("Done!")
			return True
		except:
			return False

	def remove_outliers(df,columns,n_std):
		for col in columns:
			print('Working on column: {}'.format(col))
			
			mean = df[col].mean()
			sd = df[col].std()
			
			df = df[(df[col] <= mean+(n_std*sd))]
			
		return df

	def replace_outliers(self): # Check: https://neuraldatascience.io/5-eda/data_cleaning.html
		try:
			print("Checking for outliers...")
			# Checking Z-Score < 3
			#if np.abs(stats.zscore(self.dataframe['Hum']) < 3):
			# self.dataframe['Temp_z'] = np.abs(stats.zscore(self.dataframe['Temp']) < 1)
			# self.dataframe.head()
			#outliers = np.abs(stats.zscore(self.dataframe['Hum']) < 3)
			# Replacing Outliers with Z-Score > 3
			#self.dataframe = self.dataframe.mask(self.dataframe.sub(self.dataframe.mean()).div(self.dataframe.std()).abs().gt(3))
			#self.dataframe.mask(((self.dataframe < self.dataframe.quantile(0.05)) or (self.dataframe > self.dataframe.quantile(0.95))), np.nan, axis=1)

			# # Outlier Detection Temperature
			# # IQR
			# Q1_temp = np.percentile(self.dataframe['Temp'], 25,
			#                 interpolation = 'midpoint')
			
			# Q3_temp = np.percentile(self.dataframe['Temp'], 75,
			#                 interpolation = 'midpoint')
			# IQR_temp = Q3_temp - Q1_temp
			
			# # Upper bound
			# upper = np.where(self.dataframe['Temp'] >= (Q3_temp+1.5*IQR_temp))
			# # Lower bound
			# lower = np.where(self.dataframe['Temp'] <= (Q1_temp-1.5*IQR_temp))
			
			# # Removing Outliers
			# self.dataframe['Temp'].drop(upper[0], inplace = True)
			# self.dataframe['Temp'].drop(lower[0], inplace = True)

			# # Outlier Detection Temperature
			# # IQR
			# Q1_hum = np.percentile(self.dataframe['Hum'], 25,
			#                 interpolation = 'midpoint')
			
			# Q3_hum = np.percentile(self.dataframe['Hum'], 75,
			#                 interpolation = 'midpoint')
			# IQR_hum = Q3_hum - Q1_hum
			
			# # Upper bound
			# upper = np.where(self.dataframe['Hum'] >= (Q3_hum+1.5*IQR_hum))
			# # Lower bound
			# lower = np.where(self.dataframe['Hum'] <= (Q1_hum-1.5*IQR_hum))
			
			# # Removing Outliers
			# self.dataframe['Hum'].drop(upper[0], inplace = True)
			# self.dataframe['Hum'].drop(lower[0], inplace = True)

			mean = self.dataframe['Temp'].mean()
			sd = self.dataframe['Temp'].std()
			n_std = 1
			self.dataframe = self.dataframe[(self.dataframe['Temp'] <= mean+(n_std*sd))]
			print("Temperature outliers removed...")    
			mean = self.dataframe['Hum'].mean()
			sd = self.dataframe['Hum'].std()
			n_std = 1
			self.dataframe = self.dataframe[(self.dataframe['Hum'] <= mean+(n_std*sd))]
			print("Humidity outliers removed...")          
			return True
		except:
			return False

	def check_duplicates(self):
		if duplicates:
			print("Duplicates found!")
		else:
			print("No duplicates found!")
			
	def check_timestamps(self):
		if timestamps:
			print("Incorrect timestamps found!")
		else:
			print("No incorrect timestamps found!")
			
	def check_missingvalues(self):
		if missingvalues:
			print("Missing values found!")
		else:
			print("No missing values found!")

	def check_outliers(self):
		if outliers:
			print("Outliers found!")
		else:
			print("No outliers found!")
			
	def check_sensorquality(self):
		if sensorquality:
			print("Drifting values found!")
		else:
			print("No drifting values found!")
			
	def check_wrongvalues(self):
		if wrongwalues:
			print("Wrong values found!")
		else:
			print("No wrong values found!")

	def visualize(self):
		if visualize:
			print("Visualizing Data!")
		else:
			print("Something went wrong!")
	
	def export_new_file(self):
		try:
			print("Exporting corrected file...")
			self.dataframe.to_csv('./sensor_data_temp_corrected.log', index=False)
			return True
		except:
			return False

	def start(self):
		rename_columns(self)

		

In [58]:
data_url = "./sensor_data_temp.log"
df = pd.read_csv(data_url, sep=" ", header=None, index_col=None)
df.head(1)

,0,1,2,3,4
0,1022-09-14,19:33:07,T=22.0,H=20.0,TO=45


In [59]:
test3 = CheckDataQuality(df)

In [60]:
test3.rename_columns()
test3.create_datetime()
test3.check_valid_date()
test3.replace_nat()



df[12]['Datetime']=2022-09-16 20:33:20 is not valid! Changing invalid Date to NaT. Last Date: 2022-09-14 20:28:19
df[18]['Datetime']=2022-09-14 21:03:26 is not valid! Changing invalid Date to NaT. Last Date: 2022-09-14 20:58:25
df[19]['Datetime']=2021-09-14 21:08:29 is not valid! Changing invalid Date to NaT. Last Date: 2022-09-14 21:03:26
df[24]['Datetime']=2022-09-14 21:33:32 is not valid! Changing invalid Date to NaT. Last Date: 2022-09-14 21:28:31
df[25]['Datetime']=2022-09-14 19:38:33 is not valid! Changing invalid Date to NaT. Last Date: 2022-09-14 21:33:32
df[28]['Datetime']=2022-09-14 21:53:35 is not valid! Changing invalid Date to NaT. Last Date: 2022-09-14 21:48:34
df[29]['Datetime']=2022-09-14 17:58:35 is not valid! Changing invalid Date to NaT. Last Date: 2022-09-14 21:53:35
df[162]['Datetime']=2032-09-15 09:31:05 is not valid! Changing invalid Date to NaT. Last Date: 2022-09-15 09:26:04
EXCEPTION - Something strange is going on: <class 'KeyError'>, Index: 335


False

In [61]:
#test3.change_index()
test3.format_data_columns()
test3.replace_outliers()
test3.export_new_file()

Formatting Data Columns...
String Extraction...
Dropping Column TO...
Converting from string to float...
Done!
Checking for outliers...
Temperature outliers removed...
Humidity outliers removed...
Exporting corrected file...


True

In [62]:
test3.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273 entries, 0 to 335
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Temp      273 non-null    float64       
 1   Hum       273 non-null    float64       
 2   Datetime  263 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 8.5 KB
